In [ ]:
import fnmatch
import os
import pathlib
import random
import time

import intel_extension_for_pytorch as ipex
import matplotlib.pyplot as plt
import numpy as np
import psutil
import seaborn as sns
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from tqdm import tqdm

sns.set_theme(style="whitegrid", palette="husl")

In [ ]:
simple = False
epochs = 30
ipx = True 
# JITMethod
#model_name = f"bc_resnet18_simpleFalse_IPEXTrue_30EpochsNUC"
model_name = f"bc_resnet18_simpleTrue_IPEXTrue_6Epochs_batchsz128_NUC"
model_name = f"bc_resnet18_simpleFalse_IPEXTrue_26Epochs_batchsz128_dropout0.5_NUC"

model_read = torch.jit.load(f"models/{model_name}.pt")

# STATE_DICT method
# model = models.resnet18()
# PATH = 'models/bc_resnet18_simple_NOIPEX_30Epochs_StateDict_gold.pt'
# model.fc = nn.Linear(model.fc.in_features, 3)
# model.load_state_dict(torch.load(PATH))
# model.eval()

print(model_name)

In [ ]:
imagenet_stats = [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]

class ImageFolderWithPaths(datasets.ImageFolder):
    """custom ImageFolder to get the filepaths along with the image and label data."""

    def __getitem__(self, index):
        paths = ((self.imgs[index][0]),)
        return super().__getitem__(index) + paths


def infer(model, data_path: str):
    """give trained `model` & `data_path` where images whose
    labels have to be predicted are kept.

    `data_path`: path to data eg. ./test/<random_class>/*.png
    it's important to have a folder with a`random class` name as ImgFolder
    expects it.

    returns: (
        images: images loaded from disk for inferece,
        yhats: predicted labels
        paths: image file-path on disk        )
    """
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize(*imagenet_stats)]
    )
    data = ImageFolderWithPaths(data_path, transform=transform)
    dataloader = DataLoader(
        data,
        batch_size=4,
    )
    yhats = []
    images = []
    paths = []
    for (imgs, _, fpaths) in dataloader:
        yhat = model(imgs)
        yhat = yhat.max(1)[1]
        yhat = yhat.data.cpu().numpy()
        yhats.extend(yhat)
        paths.extend(fpaths)
        images.extend(imgs.data.cpu())
    return images, yhats, paths

def show_data(dataloader, imagenet_stats=imagenet_stats, num_data=4, figsize=(10, 6)):
    """show `num_data` of images and labels from dataloader."""
    batch = next(iter(dataloader))  # batch of with images, batch of labels
    imgs, labels = (
        batch[0][:num_data],
        batch[1][:num_data].tolist(),
    )  # get num_data of images, labels
    plt.style.use("dark_background")
    _, axes = plt.subplots(1, num_data, figsize=figsize)
    for n in range(num_data):
        axes[n].set_title(labels[n])
        imgs[n] = _denormalize(imgs[n], imagenet_stats)
        axes[n].imshow(torch.clamp(imgs[n], 0, 1).permute(1, 2, 0))

def _denormalize(images, imagenet_stats):
    """de normalize dataset using imagenet std and mean to show images"""
    mean = torch.tensor(imagenet_stats[0]).reshape(1, 3, 1, 1)
    std = torch.tensor(imagenet_stats[1]).reshape(1, 3, 1, 1)
    return images * std + mean

In [ ]:
!mkdir data/ThreeClassManualRemove0s/test
!mkdir  data/ThreeClassManualRemove0s/test/unknown
!cp data/DinosaurNationalMonument/20220514/224/*.jpg data/ThreeClassManualRemove0s/test/unknown/

In [ ]:
imagenet_stats = [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]
images, yhats, img_paths = infer(
    model_read, data_path="./data/ThreeClassManualRemove0s/test/"
)
infer_dataloader = DataLoader([*zip(images, yhats)], batch_size=100, shuffle=False)
print("infered images with labels")
show_data(infer_dataloader, imagenet_stats, 20, figsize=(20, 16))


# Inference Given Large Image

In [ ]:
import glob
from PIL import Image
from os.path import exists
import os
path = "data/DinosaurNationalMonument/Dinosaur National Monument Panorama.png"
#path = "data/Explore/Pombal01.PNG"

img = Image.open(path).convert('RGB')
img.size

# Get sample near DNM Excavation

In [ ]:
left = 4*224+230
low = 4*224+230

bbox = (left, low, left + 224, low + 224)
working_slice = img.crop(bbox)
working_slice

# Simple eval function

In [ ]:
from PIL import Image
import torchvision.transforms.functional as TF

def transform_image(image):
    my_transforms = transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    return my_transforms(image).unsqueeze(0)

def eval_simple(working_slice):
    x = transform_image(working_slice)
    output = model_read(x)
    return output.detach().numpy().argmax()

eval_simple(working_slice)

# Score large map of DNM

<img src="data/DinosaurNationalMonument/Dinosaur National Monument Panorama.png" width="500"/>

We use a sliding tile (224x224) approach for scoring every pixel:
- track counts time a pixel is scored
- track cumulative score for each time a pixel is scored

In [ ]:
import glob
from PIL import Image
from os.path import exists
import os

img = Image.open(path).convert('RGB')

import time

from PIL import Image
countBuf = np.ones( (img.size[0], img.size[1]) )
sumBuf =   np.zeros( (img.size[0], img.size[1]) )

start = time.time()
step = 32 # choose a factor of 224: 1, 2, 4, 7, 8, 14, 16, 28, 32, 56, 112, and 224. Small is smooth map, large is fast
counts = {0:0, 1:0, 2:0} # inialize counts for each class
scale = {0:0, 1:1, 2:1.5}  # assign weights to predicted classes
for x in tqdm(range(0, img.size[0]-224, step)):
    for y in range(0, img.size[1]-224, step):  
        bbox = (x, y, x + 224, y + 224)
        working_slice = img.crop(bbox)
        countBuf[bbox[0]:bbox[2], bbox[1]:bbox[3]] += 1
        sumBuf[bbox[0]:bbox[2], bbox[1]:bbox[3]] += scale[eval_simple(working_slice)]
print(f"step size = {step}, Elapsed: {(time.time() - start):6.1f} sec")


# Compute the mean of scores for sliding tiles

In [ ]:
#set opacity to 159: 0 is transparent, 255 is opaque
meanBuf = sumBuf/countBuf
mat = np.uint8(meanBuf.T*159/meanBuf.max()) # scale the opacity: 0 transpartent, 255 solid
idx = mat < 0
mat[idx] = 0
output = Image.fromarray(mat)
output.save('results/bobTile.png')
np.save('results/meanBuf.npy', mat)

# Heatmap Approach

Color Legend:
- Bright Red: Bones more likely
- Bright Blue: Bone not likely

In [ ]:
from PIL import ImageDraw
import numpy as np

img = Image.open(path)
alpha = np.load('results/meanBuf.npy')

imgcv = Image.open(path.replace('.jpg','.PNG'))
heatmap_img = np.copy(imgcv)
alpha1D = alpha/alpha.max()*255

heatmap_img[:,:,0] = alpha1D
heatmap_img[:,:,1] = 255 - alpha1D
heatmap_img[:,:,2] = 255 - alpha1D

opacity = .5
imgHeat = Image.blend(Image.fromarray(heatmap_img), imgcv, alpha= opacity)
imgHeat.save('results/HeatMap.png')

print("Color Legend:\n- Red: Higher Probability\n- Blue: -Lower Probability")
imgHeat

## Notices and Disclaimers

Intel technologies may require enabled hardware, software or service activation.

No product or component can be absolutely secure. 

Your costs and results may vary. 

© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. Other names and brands may be claimed as the property of others. 